In [1]:
import serial as s
import time
import sys
import numpy as np
import pandas as pd
import csv
from __future__ import division

In [ ]:
s = serial.Serial(port='/dev/tty.usbserial', baudrate=9600)
# s2 = serial.Serial(port='/dev/tty.usbserial7', baudrate=115200)

In [ ]:
s.close()
# s2.close()

In [ ]:
cmd = '/1Z2025000R'
s.write(cmd +'\r')
t.sleep(0.2)
stri = s.read(s.inWaiting())
print stri.decode('utf8', 'ignore')

In [ ]:
cmd = '/1P896814R'
s.write(cmd +'\r')
t.sleep(0.2)
stri = s.read(s.inWaiting())
print stri.decode('utf8', 'ignore')

In [1]:
1525320/3

508440

# MOTOR
Lin Engineering 
1. Determine appropriate velocity_max = microsteps/sec
2. Determine motor limits
3. Determine conv = microsteps/mm
4. Determine orientation (P+; D-)

In [ ]:
# TODO: check zeroing
# TODO: busy
class Motor:
    def __init__(self, configFile, init=True):
        
#         f = open(configFile, 'r')
#         self.config = yaml.load(f)
        self.serial = s.Serial() # placeholder
        self.serial = s.Serial(port='/dev/tty.usbserial', baudrate=9600)
        
        self.prefix = '/1'
        self.terminator = '\r'
        self.conv = 2e6/99.24 # usteps/mm
        self.velocity_default = 1525320 # usteps/sec (75.6863784 mm/sec)
        self.velocity_limit = int(self.velocity_default/3) # usteps/sec
        self.ustep_max = 2025000 # usteps 100.48 mm
        self.ustep_min = 0

        if init:
            self.initialize()

    def initialize(self):
#         self.serial = s.Serial(**self.config['serial']) # open serial connection
        pass # TODO set moving current current
        pass # TODO set holding current
        self.set_velocity(self.velocity_limit)  # set velocity
        self.home() # move motor to home
        
    def cmd(self, cmd_string):
        full_string = self.prefix + cmd_string + self.terminator
        self.serial.write(full_string)
        repr(full_string)
        time.sleep(0.15) # TODO: monitor for response?
        return self.serial.read(self.serial.inWaiting()).decode('utf8', 'ignore')
    
#     # TEST cmd that prints full_string 
#     def cmd(self, cmd_string):
#         full_string = self.prefix + cmd_string + self.terminator
#         print full_string
    
    # velocity: (usteps/sec) 
    def set_velocity(self, velocity):
        if velocity > self.velocity_limit:
            velocity = self.velocity_limit
            print 'ERR: Desired velocity exceeds velocity_limit; velocity now set to velocity_limit'
            
        cmd_string = 'V{}R'.format(velocity)
        return self.cmd(cmd_string)
    
    def halt(self):
        cmd_string = 'T'
        self.cmd(cmd_string)
    
    def home(self):
        cmd_string = 'Z{}R'.format(self.ustep_max)
        return self.cmd(cmd_string)
    
    # absolute
    def move(self, mm):
        ustep = int(self.conv*mm)
        if ustep > self.ustep_max:
            ustep = self.ustep_max
            print 'ERR: Desired move to {} mm exceeds max of {} mm; moving to max instead'.format(mm, self.ustep_max/self.conv)
        if ustep < self.ustep_min:
            ustep = self.ustep_min
            print 'ERR: Desired move to {} mm exceeds min of {} mm; moving to min instead'.format(mm, self.ustep_min/self.conv)
            
        cmd_string = 'A{}R'.format(ustep)
        return self.cmd(cmd_string)
        
    # TODO: Try P with negative numbers?
    def move_relative(self, mm):
        ustep = int(self.conv*mm)
        ustep_current = int(self.ustep_max/2)  # TODO: limit movement (+ and -)
        
        if mm >= 0:
            if (ustep_current + ustep) > self.ustep_max:
                ustep = self.ustep_max - ustep_current
                print 'ERR: Desired move of +{} mm exceeds max of {} mm; moving to max instead'.format(mm, self.ustep_max/self.conv)
            cmd_string = 'P{}R'.format(ustep)
        
        else:
            if (ustep_current + ustep) < self.ustep_min:
                ustep = self.ustep_min - ustep_current
                print 'ERR: Desired move of {} mm exceeds min of {} mm; moving to min instead'.format(mm, self.ustep_min/self.conv)
            ustep = -1*ustep
            cmd_string = 'D{}R'.format(ustep)
        
        return self.cmd(cmd_string)
    
    def exit(self):
        self.serial.close()


In [ ]:
m = Motor('blah')

In [ ]:
m.move(23)
# time.sleep(1)
# m.move(10)

In [ ]:
m.home()

In [ ]:
print m.move(32)
time.sleep(1)
print m.move(20)

In [ ]:
print m.cmd('P100000D100000P100000D100000P100000D100000P100000D100000R')
print m.cmd('/1?0')
print m.cmd('/1?0')

In [ ]:
m.exit()

In [ ]:
print 

In [ ]:
print

# ASI Controller
Applied Scientific Instrumentation

1. Set hall effect sensors to appropriate limits
2. Determine orientation (X+-, Y+-)

In [ ]:
# TODO: Fix serial.read encoding
# TODO: better busy handling
import serial as s
import yaml
import csv
import time

class ASI_Controller:
    def __init__(self, configFile, init=True):
        f = open(configFile, 'r')
        self.config = yaml.load(f)
        self.serial = s.Serial() # placeholder
        self.plateMap = None

        if init:
            self.initialize()


    def initialize(self):
        self.serial = s.Serial(**self.config['serial']) # open serial connection

        self.cmd_xy('mc x+ y+')  # enable motor control for xy
        self.cmd_z('mc z+') # enable motor control for z

        print "Initializing stage..."
        self.m_xy(2000, -2000)  # move to switch limits (bottom right)
        self.r_xy(-0.5, 0.5)  # move from switch limits 0.5 mm

        while True:
            fp = raw_input('Type in plate map file:')
            try:
                self.loadPlateMap(fp)  # load platemap
                break
            except IOError:
                print 'No file', fp

        raw_input('Place dropper above well A1 (press enter when done)')
        self.cmd_xy('here x y')  # establish current position as 0,0

    def isBusy_xy(self):
        status = self.cmd('2h STATUS')[0]
        return status == 'B'

    def isBusy_z(self):
        status = self.cmd('1h STATUS')[0]
        return status == 'B'

    def halt(self):
        self.halt_xy()
        self.halt_z()

    def halt_xy(self):
        self.cmd_xy('HALT', False)

    def halt_z(self):
        self.cmd_z('HALT', False)

    def cmd(self, inStr):
        inStr = inStr + self.config['terminator']
        self.serial.write(inStr)
        repr(inStr)
        time.sleep(0.05)
        inBytes = self.serial.inWaiting()
        ret = self.serial.read(inBytes)
        return ret

    def cmd_xy(self, inStr, wait=True):
        while self.isBusy_xy() and wait:
            time.sleep(0.3)
        inStr = '2h ' + inStr
        return self.cmd(inStr)

    def cmd_z(self, inStr, wait=True):
        while self.isBusy_z() and wait:
            time.sleep(0.3)
        inStr = '1h ' + inStr
        return self.cmd(inStr)

    def r_xy(self, x_mm, y_mm):
        conv = self.config['conv']
        xStr = 'x=' + str(float(x_mm) * conv)
        yStr = 'y=' + str(float(y_mm) * conv)
        return self.cmd_xy(' '.join(['r', xStr, yStr]))

    def r_z(self, z_mm):
        conv = self.config['conv']
        zStr = 'z=' + str(float(z_mm) * conv)
        return self.cmd_z(' '.join(['r', zStr]))

    def move_xy(self, x_mm, y_mm):
        conv = self.config['conv']
        xStr = 'x=' + str(float(x_mm) * conv)
        yStr = 'y=' + str(float(y_mm) * conv)
        return self.cmd_xy(' '.join(['m', xStr, yStr]))

    def move_z(self, z_mm):
        conv = self.config['conv']
        zStr = 'z=' + str(float(z_mm) * conv)
        return self.cmd_z(' '.join(['m', zStr]))

    def exit(self):
        self.serial.close()





In [ ]:
a = ASI_Controller('components/controller.config')

In [ ]:
a.goToWell(92)

In [ ]:
a.r_z(-5)

In [ ]:
a.r_z(10)

In [ ]:
a.goToWell(50)

In [ ]:
a.exit()

# Autosipper

In [55]:
# I: filepath of delimited file
# P: detect delimiter/header read file accordingly
# O: list of records (no header)
def read_delim(filepath):
    f = open(filepath, 'r')
    dialect = csv.Sniffer().sniff(f.read(1024))
    f.seek(0)
    hasHeader = csv.Sniffer().has_header(f.read(1024))
    f.seek(0)
    reader = csv.reader(f, dialect)

    if hasHeader:
        reader.next()

    ret = [line for line in reader]
    return ret

def read_delim_pd(filepath):
    f = open(filepath)
    has_header = None
    if csv.Sniffer().has_header(f.read(1024)):
        has_header = 0
    f.seek(0)
    return pd.read_csv(f, header=has_header, sep=None, engine='python')


def lookup(columns, values):
    temp_df = pd.DataFrame(data=[values], columns = columns, copy=False)
    return pm.merge(temp_df, copy=False)

In [ ]:
class Autosipper():
    def __init__():
        # add ASI_Controller
        # add Motor
        XY = ASI_Controller()
        Z = Motor()
        
        while True:
            fp = raw_input('Type in plate map file:')
            try:
                self.loadPlateMap(fp)  # load platemap
                break
            except IOError:
                print 'No file', fp

        raw_input('Place dropper above well A1 (press enter when done)')
        self.cmd_xy('here x y')  # establish current position as 0,0
        
    def calibrate():
        pass
    
    def load_platemap(self, filepath):
        self.plateMap = read_delim_pd(filepath)

    def go_to_well(columns, values):
        x1,y1,z1 = np.array(lookup(columns, values)[['X','Y','Z']])

        X.home() # move needle to travel height (blocking)
        XY.move_xy(x1,y1) # move stage (blocking)
        Z.move(z1) # move needle to bottom of well (blocking)
            

# PlatemapGen

## Calibration procedure:
Could also have dedicated reference position.   
PositionMap describes the whole deck layout. 
Define XYZ limits
Define travel height as min
1. Move needle tip to some reference point (e.g. center-bottom of well A01 of top plate). Record $X_r$ $Y_r$ $Z_r$.
2. Move

Define limits of hardware

In [247]:
# I:
# row_col_num: (ct, ct) 2-tuple of 
# row_col_spacing: (mm, mm)
# well_diameter: (mm)
# O: 
def generate_platemap(row_col_num, row_col_spacing, well_diameter,):
    row_col_num = 8,12 # ct, ct
    row_col_spacing = 10,10 # mm,mm
    well_diameter = 1 # mm
    well_bottom = 
    well_top = 

    print 'n','s','r','c','name','X','Y','Z'
    for r in range(row_col_num[0]):
        for c in range(row_col_num[1]):
            n = c + (r)*row_col_num[0]
            s = ((r+1)%2)*(c + r*row_col_num[1]) + (r%2)*((r+1)*row_col_num[1] - (c+1))
            name = chr(64+r+1) + '{:02d}'.format(c+1)
            x = r*row_spacing
            y = c*col_spacing
            z = 0
            print n, s, r, c, name, x, y, z

SyntaxError: invalid syntax (<ipython-input-247-5f259df01006>, line 10)

In [ ]:
rows_cols = 8,12 # ct,ct
top_left = 0,0 # mm,mm
bottom_right = 75.0,110 # mm,mm

tuple(np.subtract(bottom_right, top_left)/(np.subtract(rows_cols, (1,1))))

# Communication: PyVISA

Install NI-VISA:
https://pyvisa.readthedocs.io/en/stable/getting_nivisa.html

In [ ]:
import visa

In [ ]:
rm = visa.ResourceManager()

In [ ]:
rm.list_resources()

In [ ]:
rm.list_resources_info()

# Documentation: Sphinx

http://thomas-cokelaer.info/tutorials/sphinx/docstring_python.html

# GUI: pywidgets.interact

http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html

In [9]:
pm = read_delim_pd('platemaps/test.csv')
pm

,N,S,r,c,name,X,Y,Z
0,0,0,0,0,A01,0,0,-
1,1,1,0,1,A02,0,10,-
2,2,2,0,2,A03,0,20,-
3,3,3,0,3,A04,0,30,-
4,4,4,0,4,A05,0,40,-
5,5,5,0,5,A06,0,50,-
6,6,6,0,6,A07,0,60,-
7,7,7,0,7,A08,0,70,-
8,8,8,0,8,A09,0,80,-
9,9,9,0,9,A10,0,90,-


In [65]:
a[['x','y','z']][a.S==23]

,X,Y,Z
12,10,0,-


In [52]:
x,y,z = np.array(go_to_well(['N'], 0))[0]

In [35]:
np.array(a)[0]

array([0, 0, '-'], dtype=object)

In [54]:
y

0